# Homework 5 - Visit the Wikipedia hyperlinks graph!

In this assignment we perform an analysis of the Wikipedia Hyperlink graph. In particular, given extra information about the categories to which an article belongs to, we are curious to rank the articles according to some criteria.

For this purpose we use the Wikipedia graph released by the SNAP group.

## Data

### 1.  Articles graph

In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
# Import dataframe
articles_graph = pd.read_csv("wiki-topcats-reduced.txt", sep = "\t", header = None, names = ["Source", "Destination"])
# Convert eveithing to string
articles_graph_str = articles_graph.applymap(str)

In [3]:
# In articles_graph df, every row is an edge, the two elements are the nodes (source and destination).
articles_graph.head()

,Source,Destination
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


In [4]:
articles_graph_str.head()

,Source,Destination
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


In [5]:
edges = list(zip(articles_graph_str.Source, articles_graph_str.Destination))

In [6]:
edges

[('52', '401135'),
 ('52', '1069112'),
 ('52', '1163551'),
 ('62', '12162'),
 ('62', '167659'),
 ('62', '279122'),
 ('62', '1089199'),
 ('62', '1354553'),
 ('62', '1400636'),
 ('62', '1403619'),
 ('62', '1537692'),
 ('62', '1544420'),
 ('64', '64873'),
 ('66', '279122'),
 ('66', '1163290'),
 ('74', '279122'),
 ('95', '1179478'),
 ('95', '1185516'),
 ('95', '1185519'),
 ('95', '1230865'),
 ('103', '107'),
 ('104', '107'),
 ('104', '1174251'),
 ('104', '1174302'),
 ('105', '1174967'),
 ('107', '104'),
 ('107', '142167'),
 ('107', '657930'),
 ('107', '1013995'),
 ('107', '1061780'),
 ('107', '1174302'),
 ('107', '1179210'),
 ('107', '1181401'),
 ('108', '104'),
 ('108', '107'),
 ('108', '1059989'),
 ('108', '1062426'),
 ('108', '1161925'),
 ('108', '1169534'),
 ('112', '107'),
 ('113', '279122'),
 ('113', '1185127'),
 ('122', '1174967'),
 ('126', '279122'),
 ('133', '279122'),
 ('133', '825464'),
 ('134', '541222'),
 ('134', '1060414'),
 ('134', '1061485'),
 ('134', '1062226'),
 ('134', '

In [7]:
# Compute the number of unique nodes
nodes = set(articles_graph["Source"]).union(set(articles_graph["Destination"]))

In [8]:
len(nodes)

461193

In [9]:
graph = defaultdict(list)
for i in range(len(edges)):
    key = edges[i][0]
    value = edges[i][1]
    graph[key].append(value)

In [10]:
# weird
len(graph.keys())

428957

In [11]:
graph

defaultdict(list,
            {'52': ['401135', '1069112', '1163551'],
             '62': ['12162',
              '167659',
              '279122',
              '1089199',
              '1354553',
              '1400636',
              '1403619',
              '1537692',
              '1544420'],
             '64': ['64873'],
             '66': ['279122', '1163290'],
             '74': ['279122'],
             '95': ['1179478', '1185516', '1185519', '1230865'],
             '103': ['107'],
             '104': ['107', '1174251', '1174302'],
             '105': ['1174967'],
             '107': ['104',
              '142167',
              '657930',
              '1013995',
              '1061780',
              '1174302',
              '1179210',
              '1181401'],
             '108': ['104', '107', '1059989', '1062426', '1161925', '1169534'],
             '112': ['107'],
             '113': ['279122', '1185127'],
             '122': ['1174967'],
             '126': ['279122'],
 

### 2. Categories

In [12]:
categories = pd.read_csv("wiki-topcats-categories.txt", sep = ";", header = None, names = ['Category', 'Documents'] )

In [13]:
categories.head()

,Category,Documents
0,Category:Buprestoidea,301 302 303 304 305 306 307 308 309 310 311 3...
1,Category:People_from_Worcester,1056 1057 1058 1059 1060 60971 76515 76871 78...
2,Category:Skin_conditions_resulting_from_physic...,971 973 1166 1167 1168 1169 1170 1171 1172 11...
3,Category:Visual_kei_bands,1297 1300 1311 1312 1313 1314 1315 1316 1319 ...
4,Category:Japanese_rock_music_groups,1297 1300 1313 1314 1315 1316 1319 1320 1322 ...


In [14]:
# CLEANING THE DATA
# Remove the eord category
categories['Category'] = categories['Category'].str.replace('Category:','')
# Remove the _ 
categories['Category'] = categories['Category'].str.replace('_',' ')
# The data inside "Documents" is a bis string: split it in to a list
categories['Documents'] = categories['Documents'].str.split()
# From string to integer
for i in range(len(categories)):
    categories["Documents"][i] = list(map(int, categories["Documents"][i]))

In [15]:
categories.head()

,Category,Documents
0,Buprestoidea,"[301, 302, 303, 304, 305, 306, 307, 308, 309, ..."
1,People from Worcester,"[1056, 1057, 1058, 1059, 1060, 60971, 76515, 7..."
2,Skin conditions resulting from physical factors,"[971, 973, 1166, 1167, 1168, 1169, 1170, 1171,..."
3,Visual kei bands,"[1297, 1300, 1311, 1312, 1313, 1314, 1315, 131..."
4,Japanese rock music groups,"[1297, 1300, 1313, 1314, 1315, 1316, 1319, 132..."


As we need the graph to analyses the categories, we are going to work only with the articles, that belongs to "articles_grap" i.e with the "nodes_set" set.

In [16]:
# Keep only the intersection between unique_docs and the list of documents
intersection = []
for i in range(len(categories)):
    inter = set(categories["Documents"][i]).intersection(nodes)
    intersection.append(inter)
categories['Documents'] = intersection

In [17]:
categories.head()

,Category,Documents
0,Buprestoidea,{}
1,People from Worcester,"{1194242, 84354, 537220, 89734, 85767, 1178634..."
2,Skin conditions resulting from physical factors,{}
3,Visual kei bands,{604876}
4,Japanese rock music groups,"{604876, 1759782}"


We are also going to work with the categories that have more than 3.500 articles. For achieve that aim, we computed the length of each list of articles, and keep only the ones that have more than 3.500 articles.

In [18]:
# Compute the length of each category 
leng = []
for i in range(len(categories)):
    l = len(categories["Documents"][i])
    leng.append(l)
# Apending a column in our df
categories['Length'] = leng
# Selecting only the rows that we care about
categories = categories[categories["Length"] > 3500]

In [19]:
categories.head()

,Category,Documents,Length
868,English footballers,"{622642, 622644, 622647, 622649, 557114, 62266...",7538
869,The Football League players,"{1671189, 622644, 622647, 622649, 557114, 1245...",7814
876,Association football forwards,"{81922, 90119, 73736, 81928, 1359881, 81930, 7...",5097
898,Association football goalkeepers,"{737292, 737293, 1671183, 73743, 106515, 10651...",3737
900,Association football midfielders,"{1671168, 884755, 884774, 622637, 1671214, 167...",5827


In [20]:
# Keeping only the columns that we need
columns_of_interest = ["Category", "Documents"]
categories = categories[columns_of_interest]
# Reset the index
categories = categories.reset_index(drop = True)
# Make every element of "Documents", a list (from a set)
for i in range(len(categories)):
    categories["Documents"][i] = list(categories["Documents"][i])

In [21]:
# From integer to string
for i in range(len(categories)):
    categories["Documents"][i] = list(map(str, categories["Documents"][i]))

In [26]:
type(categories["Documents"][0][0])

str

In [27]:
# Create a dictionary with the categories nedeed
categories_dict = dict(zip(categories.Category, categories.Documents))

In [29]:
categories_dict

{'English footballers': ['622642',
  '622644',
  '622647',
  '622649',
  '557114',
  '622661',
  '622678',
  '622686',
  '622687',
  '622690',
  '622695',
  '1638547',
  '1507499',
  '819483',
  '1638693',
  '1507666',
  '622949',
  '1507753',
  '885208',
  '885215',
  '885217',
  '164330',
  '1278447',
  '1049095',
  '721426',
  '1671712',
  '1671715',
  '1671716',
  '1671717',
  '1081895',
  '1671773',
  '852653',
  '852655',
  '1671949',
  '1573653',
  '721713',
  '525190',
  '525212',
  '1573874',
  '132132',
  '820267',
  '656440',
  '1246283',
  '164942',
  '459888',
  '33973',
  '623841',
  '623848',
  '623849',
  '656809',
  '1705426',
  '656901',
  '591394',
  '1082953',
  '624256',
  '1345275',
  '132887',
  '132916',
  '132918',
  '821064',
  '1378126',
  '526306',
  '526322',
  '559143',
  '297036',
  '231584',
  '1640635',
  '952599',
  '952600',
  '1345871',
  '395610',
  '1640795',
  '395622',
  '395623',
  '395634',
  '395644',
  '395654',
  '395709',
  '395714',
  '395

In [28]:
# Info of our 29 surviving categories
i = 1
for key in categories_dict.keys():
    print(i, key, len(categories_dict[key]))
    i += 1

1 English footballers 7538
2 The Football League players 7814
3 Association football forwards 5097
4 Association football goalkeepers 3737
5 Association football midfielders 5827
6 Association football defenders 4588
7 Living people 348300
8 Harvard University alumni 5549
9 Major League Baseball pitchers 5192
10 Members of the United Kingdom Parliament for English constituencies 6491
11 Indian films 5568
12 Year of death missing 4122
13 Year of birth missing (living people) 28498
14 Rivers of Romania 7729
15 Main Belt asteroids 11660
16 Asteroids named for people 4895
17 English-language albums 4760
18 British films 4422
19 English-language films 22463
20 American films 15159
21 People from New York City 4614
22 American television actors 11531
23 American film actors 13865
24 Debut albums 7561
25 Black-and-white films 10759
26 Year of birth missing 4346
27 Place of birth missing (living people) 5532
28 American military personnel of World War II 3720
29 Windows games 4025


### 3. Pages Names

In [30]:
pages_names = pd.read_csv("wiki-topcats-page-names.txt", sep = "\t", header = None)

In [31]:
pages_names.head()

,0
0,0 Chiasmal syndrome
1,1 Kleroterion
2,2 Pinakion
3,3 LyndonHochschildSerre spectral sequence
4,4 Zariski's main theorem


## [RQ1] 
###  Build the graph $G=(V, E)$, where $V$ is the set of articles and $E$ the hyperlinks among them, and provide its basic information:

In [32]:
edges

[('52', '401135'),
 ('52', '1069112'),
 ('52', '1163551'),
 ('62', '12162'),
 ('62', '167659'),
 ('62', '279122'),
 ('62', '1089199'),
 ('62', '1354553'),
 ('62', '1400636'),
 ('62', '1403619'),
 ('62', '1537692'),
 ('62', '1544420'),
 ('64', '64873'),
 ('66', '279122'),
 ('66', '1163290'),
 ('74', '279122'),
 ('95', '1179478'),
 ('95', '1185516'),
 ('95', '1185519'),
 ('95', '1230865'),
 ('103', '107'),
 ('104', '107'),
 ('104', '1174251'),
 ('104', '1174302'),
 ('105', '1174967'),
 ('107', '104'),
 ('107', '142167'),
 ('107', '657930'),
 ('107', '1013995'),
 ('107', '1061780'),
 ('107', '1174302'),
 ('107', '1179210'),
 ('107', '1181401'),
 ('108', '104'),
 ('108', '107'),
 ('108', '1059989'),
 ('108', '1062426'),
 ('108', '1161925'),
 ('108', '1169534'),
 ('112', '107'),
 ('113', '279122'),
 ('113', '1185127'),
 ('122', '1174967'),
 ('126', '279122'),
 ('133', '279122'),
 ('133', '825464'),
 ('134', '541222'),
 ('134', '1060414'),
 ('134', '1061485'),
 ('134', '1062226'),
 ('134', '

In [33]:
nodes

{52,
 62,
 64,
 66,
 74,
 95,
 96,
 103,
 104,
 105,
 107,
 108,
 112,
 113,
 117,
 122,
 126,
 133,
 134,
 136,
 137,
 153,
 154,
 155,
 156,
 158,
 159,
 160,
 163,
 166,
 167,
 170,
 173,
 174,
 185,
 186,
 190,
 195,
 204,
 209,
 212,
 214,
 215,
 227,
 256,
 845,
 847,
 928,
 931,
 935,
 936,
 938,
 940,
 943,
 948,
 952,
 963,
 964,
 979,
 980,
 1024,
 1027,
 1028,
 1030,
 1032,
 1035,
 1038,
 1040,
 1046,
 1048,
 1049,
 1050,
 1054,
 1055,
 1058,
 1062,
 1068,
 1070,
 1073,
 1083,
 1084,
 1086,
 1087,
 1088,
 1089,
 1098,
 1099,
 1100,
 1101,
 1102,
 1104,
 1106,
 1109,
 1114,
 1115,
 1116,
 1118,
 1119,
 1120,
 1121,
 1122,
 1123,
 1124,
 1125,
 1126,
 1127,
 1128,
 1129,
 1131,
 1132,
 1133,
 1134,
 1135,
 1136,
 1137,
 1138,
 1143,
 1144,
 1145,
 1146,
 1147,
 1148,
 1149,
 1150,
 1151,
 1152,
 1153,
 1154,
 1157,
 1158,
 1162,
 1163,
 1178,
 1200,
 1202,
 1231,
 1232,
 1267,
 1271,
 1285,
 1286,
 1287,
 1289,
 1301,
 1302,
 1318,
 1344,
 1347,
 1370,
 1394,
 1398,
 1401,
 14

In [34]:
graph

defaultdict(list,
            {'52': ['401135', '1069112', '1163551'],
             '62': ['12162',
              '167659',
              '279122',
              '1089199',
              '1354553',
              '1400636',
              '1403619',
              '1537692',
              '1544420'],
             '64': ['64873'],
             '66': ['279122', '1163290'],
             '74': ['279122'],
             '95': ['1179478', '1185516', '1185519', '1230865'],
             '103': ['107'],
             '104': ['107', '1174251', '1174302'],
             '105': ['1174967'],
             '107': ['104',
              '142167',
              '657930',
              '1013995',
              '1061780',
              '1174302',
              '1179210',
              '1181401'],
             '108': ['104', '107', '1059989', '1062426', '1161925', '1169534'],
             '112': ['107'],
             '113': ['279122', '1185127'],
             '122': ['1174967'],
             '126': ['279122'],
 

#### 1. Direct graph
Yes

#### 2. Number of nodes

In [35]:
N = len(nodes)
N

461193

####  3. Number of edges

In [36]:
E = len(edges)
E

2645247

#### 4.  Average node degree

The average degree, in the case of directed graphs is defined by: 
${\displaystyle \langle k\rangle ={\tfrac {E}{N}}}$. Were $E$ is the number of edges, and $N$ de number of nodes.

In [37]:
k = E/N
round(k,2)

5.74

#### 5. Density

For directed graphs, the graph density is defined as:

${\displaystyle D={\frac {E}{N\,(N-1)}}}$

Where $E$ is the number of edges and $N$ is the number of nodes in the graph. 

A dense graph is a graph which the number of edges is close to the maximal number of edges. The opposite, a graph with only a few edges, is a sparse graph. 

In [38]:
D = E/(N*(N-1))
D

1.2436602635647606e-05

As we can see, this is a sparse graph since the density measurement is close to zero.

## [RQ2] 
### Given a category $C_0 = \{article_1, article_2, \dots \}$ as input we want to rank all of the nodes in $N$.

In [39]:
C0 = input()

English footballers


### Block-ranking

We define de block ranking as:
$ block_{RANKING} =\begin{bmatrix} C_0 \\ C_1 \\ \dots \\ C_c\\ \end{bmatrix}$

The first category of the rank, $C_0$, always corresponds to the input category. The order of the remaining categories is given by:

$distance(C_0, C_i) = median(ShortestPath(C_0, C_i))$

The lower is the distance from $C_0$, the higher is the $C_i$ position in the rank. $ShortestPath(C_0, C_i)$ is the set of all the possible shortest paths between the nodes of $C_0$ and $C_i$. 

Moreover, the length of a path is given by the sum of the weights of the edges it is composed by.

### Functions
#### 1. Breadth First Search (BFS): previous node

In [40]:
def BFS_prev(graph, S):
    prev = {}
    visited = []
    Q = []
    Q.append(S)
    while Q:
        current = Q.pop()
        for child in graph[current]:
            if child not in visited: # havent been discovered jet
                Q = Q + list(child)
                visited.append(child)
                prev[child] = current
    return prev

In [142]:
BFS_prev(graph, "52")

{'401135': '52', '1069112': '52', '1163551': '52'}

#### 2. Shortest Path between two nodes

In [140]:
def ShortestPath(S, G, graph):
    D = BFS_prev(graph, S)
    path = []
    if S == G:
        return len(path) 
    elif G not in D:
        pass
    else:
        C = G
        while C != S:
            path.append(D[C])
            C = D[C]
        return len(path)

In [197]:
ShortestPath("52", "1069112", graph)

1

#### 3. Categories score function

In [145]:
import numpy as np
from numpy import nanmedian

In [198]:
def ScoreCategories(C, D):
    cat_0 = categories_dict[C]
    cat_1 = categories_dict[D] 
    result = []
    for article_0 in cat_0:
        for article_1 in cat_1:
            sp = ShortestPath(article_0, article_1, graph)
            if sp is not None:
                result.append(sp)
    return np.median(result)

#### 4. Block-Ranking Dictionary

In [170]:
import operator

In [ ]:
ranking_dict = {}
for key in categories_dict.keys():
    if key != C0:
        score = ScoreCategories(C0, key)
        ranking_dict[key] = score

In [214]:
ranking_dict

{'The Football League players': 1.0,
 'Association football forwards': 1.0,
 'Association football goalkeepers': 1.0,
 'Association football midfielders': 1.0,
 'Association football defenders': 1.0,
 'Living people': 1.0,
 'Harvard University alumni': 1.0,
 'Major League Baseball pitchers': nan,
 'Members of the United Kingdom Parliament for English constituencies': 1.0,
 'Indian films': 1.0,
 'Year of death missing': 1.0,
 'Year of birth missing (living people)': 1.0,
 'Rivers of Romania': nan,
 'Main Belt asteroids': nan,
 'Asteroids named for people': nan,
 'English-language albums': 1.0,
 'British films': 1.0,
 'English-language films': 1.0,
 'American films': 1.0,
 'People from New York City': 1.0,
 'American television actors': 1.0,
 'American film actors': 1.0,
 'Debut albums': 1.0,
 'Black-and-white films': 1.0,
 'Year of birth missing': 1.0,
 'Place of birth missing (living people)': 1.0,
 'American military personnel of World War II': 1.0,
 'Windows games': 1.0}

In [234]:
for key in ranking_dict.keys():
    if ranking_dict[key] != 1.0:
        ranking_dict[key] = 999

In [235]:
ranking_dict

{'The Football League players': 1.0,
 'Association football forwards': 1.0,
 'Association football goalkeepers': 1.0,
 'Association football midfielders': 1.0,
 'Association football defenders': 1.0,
 'Living people': 1.0,
 'Harvard University alumni': 1.0,
 'Major League Baseball pitchers': 999,
 'Members of the United Kingdom Parliament for English constituencies': 1.0,
 'Indian films': 1.0,
 'Year of death missing': 1.0,
 'Year of birth missing (living people)': 1.0,
 'Rivers of Romania': 999,
 'Main Belt asteroids': 999,
 'Asteroids named for people': 999,
 'English-language albums': 1.0,
 'British films': 1.0,
 'English-language films': 1.0,
 'American films': 1.0,
 'People from New York City': 1.0,
 'American television actors': 1.0,
 'American film actors': 1.0,
 'Debut albums': 1.0,
 'Black-and-white films': 1.0,
 'Year of birth missing': 1.0,
 'Place of birth missing (living people)': 1.0,
 'American military personnel of World War II': 1.0,
 'Windows games': 1.0}

In [236]:
L = sorted(ranking_dict, key=ranking_dict.__getitem__)
ranking = [C0]
ranking = ranking + L
ranking

['English footballers',
 'The Football League players',
 'Association football forwards',
 'Association football goalkeepers',
 'Association football midfielders',
 'Association football defenders',
 'Living people',
 'Harvard University alumni',
 'Members of the United Kingdom Parliament for English constituencies',
 'Indian films',
 'Year of death missing',
 'Year of birth missing (living people)',
 'English-language albums',
 'British films',
 'English-language films',
 'American films',
 'People from New York City',
 'American television actors',
 'American film actors',
 'Debut albums',
 'Black-and-white films',
 'Year of birth missing',
 'Place of birth missing (living people)',
 'American military personnel of World War II',
 'Windows games',
 'Major League Baseball pitchers',
 'Rivers of Romania',
 'Main Belt asteroids',
 'Asteroids named for people']